<a href="https://colab.research.google.com/github/grfaith/AmStory/blob/main/Finding_explore_kw_proximity_hits_in_AS_string_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###Installs

!pip install datasets
!pip install ipympl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 813.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.8 MB/s et

This is code for looking at a range of years in the American Stories data and finding articles which appear with the string 'explor' and saving their information to disk.

In [2]:
#Imports
import json
import pandas as pd
from datasets import load_dataset
import tqdm as tq
from google.colab import files
import re

kw_distance = 150


In [3]:
# Full ChronAm extends back to 1774 (I think). Previous searches have returnedo hits
start_year = 1820 #This is start-year inclusive

# Complete dataset runs to 1940 inclusive, but this particular implementation runs out of space about 1905, so I'm breaking it into chunks
end_year = 1905 #This is end-year exclusive (I think)

In [4]:
### Cell for loading files from local drive
kw_file = files.upload()

# Specify custom column names
column_names = ["kword", "kwyear", "kwtype"]

# Read the uploaded CSV file into a DataFrame with custom column names
for fn in kw_file.keys():
    kw_df = pd.read_csv(fn, names=column_names, header=None)

# Display information about the uploaded file
for fn in kw_file.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(kw_file[fn])))



Saving AS_Explore_KW_String_and_Word.csv to AS_Explore_KW_String_and_Word.csv
User uploaded file "AS_Explore_KW_String_and_Word.csv" with length 455 bytes


In [5]:
# Defining function to load dataset

def load_text_dataset(dataset_year_str):
    """
    This function pulls a dataset of a specific year from the HuggingFace Hub.

    Parameters:
        dataset_year (int): The year of the dataset to be pulled..

    Returns:
        dataset_article_level: dataset for appropriate year
    """
    # Download data for the dataset year at the associated article level (Default)
    # dataset = load_dataset("dell-research-harvard/AmericanStories", "subset_years", year_list=[dataset_year])

    # now let's load our data, we have to specify the huggingface location of our
    # data, the fact that we want to have a subset of years, and our desired years
    dataset_article_level=load_dataset("dell-research-harvard/AmericanStories",
                                      "subset_years",
                                       year_list=[dataset_year_str],
                                       trust_remote_code=True
                                       )

    return dataset_article_level

In [6]:
### Function to filter kw data in use based on years of discovery in kw file.

def get_kw(dataset_year_str):
    """
    This function loads a CSV file to create a DataFrame and filters out keywords where the second column is less than 1774
    Parameters:
        kw_file loaded from prompt above
    Returns:
        pandas.DataFrame: The filtered Data
    """
    # Convert dataset_year to integer
    dataset_year = int(dataset_year_str)

    # Filter the rows based on the condition
    kw_df_filter = kw_df[kw_df['kwyear'] <= dataset_year]

    # print(kw_df_filter)

    return kw_df_filter



In [7]:

def process_kw(dataset_year_str, kw_df_filter, dataset_article_level):
    """
    This function processes words in a DataFrame.

    Parameters:
        kw_df_filter (pandas.DataFrame): The DataFrame containing keywords
        dataset_article (DatasetDict): A dictionary-like object containing datasets for different years.

    Returns:
        dataset_year_hits (DataFrame or None): A DataFrame containing results if found, or None if no results were found.
    """
    print ("Searching within ", dataset_year_str)

    # Creating an empty dataframe
    current_year_df = pd.DataFrame()

    for index, row in kw_df_filter.iterrows():
        explore_kw = row.iloc[0]
        kw_type = row.iloc[2]
        # print(explore_kw, kw_type)
        result_df = kw_pair_search(dataset_article_level, dataset_year_str, explore_kw, kw_distance, kw_type)
        # Concatenate the single search result onto the results DataFrame
        current_year_df = pd.concat([current_year_df, result_df], ignore_index=True)

    return current_year_df


In [8]:
## key word search within articles to find keywords with kw_distance of the string 'explor'

def kw_pair_search(dataset_article, dataset_year, explor_kw, kw_distance, kw_type):
    """
    This function searches through the dataset, by kw, to look for matching articles.

    Parameters:
        dataset_article (DatasetDict): A dictionary-like object containing datasets for different years.
        dataset_year (int): The year of the dataset.
        explor_kw (str): The keyword to search for.
        kw_distance (int): The maximum distance allowed between occurrences of the two keywords.

    Returns:
        df_of_articles_containing_two_words (DataFrame): A DataFrame containing article IDs and article texts of articles containing both keywords within the specified distance.
    """
    # Access the dataset for the specific year
    year_dataset = dataset_article[dataset_year]

    # Access the 'raw_data_string' column
    articles = year_dataset['article']

    # Create empty list to store matching articles
    articles_containing_two_words = []

    for article_n, article_text in enumerate(articles):
        article_text = article_text.lower()
        if "explor" in article_text and explor_kw in article_text:
            # print ("We have a hit at", article_n)

            # Define the pattern to search for "explor" within words
            pattern_explor = re.compile(r'\b\w*explor\w*\b')

            # Find indices of matches in the article text
            explor_indices = [match.start() for match in pattern_explor.finditer(article_text)]

            # Determine if the search needs to be on a string or on a word

            if kw_type == "string":
                # Define the pattern to search for the value of 'explor_kw' within words
                pattern_kw = re.compile(r'\b\w*' + re.escape(explor_kw) + r'\w*\b', flags=re.IGNORECASE)
            elif kw_type == "word":
                # Define the pattern to search for the exact value of 'explor_kw' as a whole word
                pattern_kw = re.compile(r'\b' + re.escape(explor_kw) + r'\b', flags=re.IGNORECASE)
            else:
                # Handle the case when kw_type is neither "string" nor "word"
                raise ValueError("Invalid kw_type. Must be either 'string' or 'word'.")


            # Find indices of matches in the article text
            kw_indices = [match.start() for match in pattern_kw.finditer(article_text)]
            #print (kw_indices)
            #input ("Press Enter to continue")

            # Check if there are any pairs of indices within the specified distance
            for explor_index in explor_indices:
                for kw_index in kw_indices:
                    if abs(explor_index - kw_index) <= kw_distance:
                        # print ("SUCCESS!!", dataset_year, article_n)
                        # Append the matching article information to the list
                        articles_containing_two_words.append({
                            'article_year': dataset_year,
                            'row_number': article_n,
                            'article_ID': dataset_article[dataset_year][article_n]["article_id"],
                            'newspaper_name': dataset_article[dataset_year][article_n]["newspaper_name"],
                            'edition': dataset_article[dataset_year][article_n]["edition"],
                            'date': dataset_article[dataset_year][article_n]["date"],
                            'page': dataset_article[dataset_year][article_n]["page"],
                            'headline': dataset_article[dataset_year][article_n]["headline"],
                            'byline': dataset_article[dataset_year][article_n]["byline"],
                            'article': dataset_article[dataset_year][article_n]["article"],
                            'keyword_hit': explor_kw,
                            'kw_index': kw_index,
                            'explor_index': explor_index,
                        })
                        # Once a matching pair is found, break out of the loop
                        # break
                    else:
                        continue

    # Convert the list of dictionaries to a DataFrame
    df_of_articles_containing_two_words = pd.DataFrame(articles_containing_two_words)
    return df_of_articles_containing_two_words


In [9]:
results = pd.DataFrame()

for loop_year in range(start_year, end_year):
    dataset_year_str = str(loop_year)
    try:
        # Load dataset for current loop_year
        dataset_article_level = load_text_dataset(dataset_year_str)

        # Get valid keyword filters for current year
        kw_df_filter = get_kw(loop_year)
        # print ("Processing year", dataset_year_str)

        # Process keyword filters and dataset for current loop year to get hits
        year_search_result = process_kw(dataset_year_str,kw_df_filter, dataset_article_level)

        # if not year_search_result.empty:
        #   print("Found: ", year_search_result)
        # else:
        #   print("No results found for ", dataset_year_str)

        # Concatenate the single search result onto the results DataFrame
        results = pd.concat([results, year_search_result])

    except ValueError:
        print(f"Dataset empty for {dataset_year_str}. Moving to the next year.")
        continue

results = results.reset_index(drop=True)
results.to_csv('AS_Explor_KW_Hits_Prox_SW_Part_1_In Fxn.csv', index=False)
files.download("AS_Explor_KW_Hits_Prox_SW_Part_1_In Fxn.csv")
print("Finished")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1820': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1820.tar.gz'}


Generating 1820 split: 0 examples [00:00, ? examples/s]

Loading associated
Searching within  1820
Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1821': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1821.tar.gz'}


Generating 1821 split: 0 examples [00:00, ? examples/s]

...

Generating 1905 split: 0 examples [00:00, ? examples/s]

Loading associated
Error loading file: mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1905/1905-04-22_p1_sn86091092_00295869367_1905042201_1214.json
Error loading file: mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1905/1905-01-21_p1_sn86091092_00295869367_1905012101_1106.json
Error loading file: mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1905/1905-06-17_p1_sn86091092_00295869367_1905061701_1282.json
Error loading file: mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1905/1905-01-14_p1_sn86091092_00295869367_1905011401_1098.json
Error loading file: mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1905/1905-02-11_p1_sn86091092_00295869367_1905021101_1130.json
Searching within  1905
Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1906': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1906.tar.

Generating 1906 split: 0 examples [00:00, ? examples/s]

Loading associated


DatasetGenerationError: An error occurred while generating the dataset

In [1]:
# Run this cell if you have to interrupt the above loop to save progress.
#  Update file names as appropriate.


results = results.reset_index(drop=True)
results.to_csv('AS_Explor_KW_Hits_Prox_SW_Part_1.csv', index=False)
files.download("AS_Explor_KW_Hits_Prox_SW_Part_1.csv")



NameError: name 'results' is not defined

In [ ]:
print ("Now save output file to local disk")
input("Press Enter to continue...")


# **SOLVENT FRONT**

In [ ]:
print(results.info)


In [ ]:
# let's start with deciding which years we want data for
scan_level_desired_years = ["1900",]

# now let's load our data, we have to specify the huggingface location of our
# data, the fact that we want to have a subset of years, and our desired years
dataset_scan_level=load_dataset("dell-research-harvard/AmericanStories",
                                "subset_years_content_regions",
                                year_list=scan_level_desired_years
                                )

In [ ]:
print(dataset_article_level['1900'][377385]["article"])